<a href="https://colab.research.google.com/github/yongbin02/P---AI/blob/main/Kmeans_UMAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import umap
import joblib  # 모델 저장용
import json    # 매핑 파일 저장용

### 1. 경로 및 설정

In [5]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/Project_P/'
TRAIN_FILE = BASE_PATH + 'data/감성대화말뭉치(최종데이터)_Training.xlsx'
VAL_FILE = BASE_PATH + 'data/감성대화말뭉치(최종데이터)_Validation.xlsx'
NUM_CLUSTERS = 12 # 군집 개수

### 2. 데이터 로드 및 전처리

In [17]:
def load_and_preprocess(filepath):
    print(f"\n📂 파일 읽는 중... : {filepath}")
    try:
        df = pd.read_excel(filepath, engine='openpyxl')
    except Exception as e:
        print(f"❌ 파일 읽기 실패: {e}")
        return None

    # 필요한 컬럼 확인
    required_cols = ['사람문장1', '사람문장2', '사람문장3']
    target_cols = [col for col in required_cols if col in df.columns]

    if not target_cols:
        print(f"❌ 중요 컬럼이 없습니다.")
        return None

    # 수정 부분
    # Melt 대신 Apply 사용
    # NaN(빈칸)이 있을 수 있으므로 처리하며 병합
    def merge_sentences(row):
        texts = []
        for col in target_cols:
            sentence = str(row[col]).strip()
            # 'nan'이나 빈 문자열이 아니면 리스트에 추가
            if sentence and sentence.lower() != 'nan':
                texts.append(sentence)
        return " ".join(texts) # 띄어쓰기로 이어 붙임

    # 'text'라는 새로운 컬럼에 합친 문장
    df['text'] = df.apply(merge_sentences, axis=1)

    # 합쳤는데도 내용이 없으면 제거
    df_clean = df.dropna(subset=['text'])
    df_clean = df_clean[df_clean['text'].str.strip() != ""]

    # 필요한 컬럼만 남기고 리턴 (감정 라벨은 검증용으로 유지)
    return df_clean[['감정_대분류', 'text']]

In [18]:
print("--- 1. 데이터 로드 ---")
train_df = load_and_preprocess(TRAIN_FILE)
val_df = load_and_preprocess(VAL_FILE)

if train_df is None:
    print("❌ 데이터 로드 실패")
    exit()

print(f"✅ Training 데이터: {len(train_df)}개")
print(f"✅ Validation 데이터: {len(val_df)}개")

--- 1. 데이터 로드 ---

📂 파일 읽는 중... : /content/drive/MyDrive/Colab Notebooks/Project_P/data/감성대화말뭉치(최종데이터)_Training.xlsx

📂 파일 읽는 중... : /content/drive/MyDrive/Colab Notebooks/Project_P/data/감성대화말뭉치(최종데이터)_Validation.xlsx
✅ Training 데이터: 51630개
✅ Validation 데이터: 6641개


### 3. SBERT 임베딩 (벡터화)

In [ ]:
print("\n--- 2. SBERT 임베딩 시작 ---")
model = SentenceTransformer('jhgan/ko-sbert-multitask')
train_vectors = model.encode(train_df['text'].tolist(), show_progress_bar=True)

### 4. UMAP 차원 축소

In [20]:
print(f"\n--- 3. UMAP 차원 축소 시작 (768차원 -> 5차원) ---")
# n_components=5: 군집화를 위해 적당히 정보를 남겨둠 (시각화용은 보통 2)
# n_neighbors=15: 주변 데이터를 얼마나 볼 것인지 (기본값)
# min_dist=0.0: 데이터를 빡빡하게 뭉치게 해서 군집화 잘되게 함
reducer = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# 학습 데이터 변환 (Fit & Transform)
train_umap = reducer.fit_transform(train_vectors)
print("✅ UMAP 변환 완료!")


--- 3. UMAP 차원 축소 시작 (768차원 -> 5차원) ---


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


✅ UMAP 변환 완료!


### 5. K-Means 군집화 (UMAP 데이터)

In [21]:
print(f"\n--- 4. K-Means 군집화 시작 (K={NUM_CLUSTERS}) ---")
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init='auto')
kmeans.fit(train_umap) # UMAP 사용

# 데이터에 군집 번호 붙이기
train_df['cluster'] = kmeans.labels_


--- 4. K-Means 군집화 시작 (K=12) ---


### 6. 결과 분석

In [22]:
print("\n--- 5. 군집별 내용 분석 ---")
cluster_analysis = []

for i in range(NUM_CLUSTERS):
    cluster_data = train_df[train_df['cluster'] == i]
    if len(cluster_data) == 0: continue

    dominant_emotion = cluster_data['감정_대분류'].value_counts().idxmax()
    emotion_percent = (cluster_data['감정_대분류'].value_counts().max() / len(cluster_data)) * 100

    num_samples = min(3, len(cluster_data))
    examples = cluster_data['text'].sample(num_samples, random_state=42).values

    print(f"--- [Cluster {i}] 주된 감정: {dominant_emotion} ({emotion_percent:.1f}%) ---")
    for idx, ex in enumerate(examples):
        print(f"   예시 {idx+1}: {ex}")
    print("-" * 50)

    cluster_analysis.append({
        'cluster_id': i,
        'dominant_emotion': dominant_emotion,
        'purity': round(emotion_percent, 2),
        'examples': examples.tolist()
    })


--- 5. 군집별 내용 분석 ---
--- [Cluster 0] 주된 감정: 당황 (19.9%) ---
   예시 1: 아이들이 돈도 없는데 맛있는 음식을 먹고 싶다고 계속 졸라서 화를 했어. 돈이 없어도 이런 것 때문에 아이들을 화내면 안 되는데 말이야. 아이들에게 사과하고 조금씩이라도 돈을 모아서 한 번쯤은 맛있는 걸 사줘야겠어.
   예시 2: 아이가 갑자기 학교를 자퇴하겠대. 학교에서 배울 게 없대. 나가서 돈 벌고 싶다고. 철없는 생각이라고 말했지만 진짜 그럴까 봐 걱정도 커. 그 이후로는 서로 대화를 피하고 있어.
   예시 3: 난 그냥 세금 받아 놀고먹는 사람들 정말 싫었는데. 어. 근데 나도 나이 들면 그 욕하던 무리 중의 하나가 될 거 같아서 요즘 나 자신이 싫어. 모아둔 돈이 없어. 나라가 보조해 주는 돈이 없으면 기본생활조차 못 할 수도 있을 거 같아.
--------------------------------------------------
--- [Cluster 1] 주된 감정: 상처 (21.1%) ---
   예시 1: 친구가 모임 회비는 내지도 않으면서 자꾸 옷은 사서 입는 걸 보면 너무 화가 나. 옷은 사면서 왜 회비는 안 낼까? 정말 화가 나. 대화로 풀었어. 다시 한번 친구랑 대화를 해봐야겠어.
   예시 2: 요즘 코로나 때문에 자주 못 만나고 단톡방 대화를 많이 하는데 오히려 더 친구들 사이가 좋아진 느낌이야. 귀찮다고 단톡방에 참여 안 하던 친구들도 코로나 이후에 다른 모임이 줄어서 그런지 단톡방 대화에라도 열심이더라고. 나도 그렇지만 다른 친구들도 나름 만족하고 있는 거 같더라고. 단톡방 활성화가 굉장히 좋은 영향을 미치고 있는 거 같아.
   예시 3: 오늘 버스에서 벨을 못 눌러 내려야 하는 정거장을 지나쳤어. 내가 눌러야 하는 상황에 남들의 관심이 나에게 오는 게 싫거든. 마음만은 내 성격을 바꾸고 싶은데 잘 안 돼.
--------------------------------------------------
-

### 7. Validation 검증 (UMAP 적용)

In [23]:
print("\n--- 6. Validation 데이터 검증 ---")
# 1) SBERT 임베딩
val_vectors = model.encode(val_df['text'].tolist(), show_progress_bar=False)
# 2) UMAP 변환 (Fit 하지 않고 Transform만)
val_umap = reducer.transform(val_vectors)
# 3) K-Means 예측
val_df['predicted_cluster'] = kmeans.predict(val_umap)
print("✅ Validation 완료!")


--- 6. Validation 데이터 검증 ---
✅ Validation 완료!


### 8. 파일 저장

In [24]:
print("\n--- 7. 파일 저장 (구글 드라이브) ---")

# (1) 데이터 저장
train_df.to_excel(BASE_PATH + 'data/clustered_train_result_umap.xlsx', index=False)
print(f"✅ 분석 데이터 저장됨")

# (2) K-Means 모델 저장
joblib.dump(kmeans, BASE_PATH + 'data/kmeans_model.pkl')
print(f"✅ K-Means 모델 저장됨")

# (3) UMAP 모델 저장 -> kmeans와 함께 백엔드 서빙
joblib.dump(reducer, BASE_PATH + 'data/umap_model.pkl')
print(f"✅ UMAP 모델 저장됨: umap_model.pkl (백엔드 서빙)")

# (4) 요약 정보
with open(BASE_PATH + 'cluster_summary_with_UMAP.json', 'w', encoding='utf-8') as f:
    json.dump(cluster_analysis, f, ensure_ascii=False, indent=4)
print(f"✅ 요약 정보 저장됨")

print("\n🎉 UMAP 적용 완료")


--- 7. 파일 저장 (구글 드라이브) ---
✅ 분석 데이터 저장됨
✅ K-Means 모델 저장됨
✅ UMAP 모델 저장됨: umap_model.pkl (백엔드 서빙)
✅ 요약 정보 저장됨

🎉 UMAP 적용 완료


In [ ]:
# 파일은 kmeans_model.pkl과 umap_model.pkl 2개임
# 사용자 입력 -> SBERT -> umap_model.transform(벡터) -> kmeans_model.predict(압축된 벡터)
# 순서로 서빙해야 에러 안남